In [4]:
import os
import pandas as pd

# Input and output paths
portfolio_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\01_Input\06_PortFolio\05_Port_Month_Year_FullData"
output_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\02_Output\12_BAB_Test_weight\BAB_data"
summary_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\02_Output\12_BAB_Test_weight\BAB_sum_data"

# Ensure output directories exist
os.makedirs(output_path, exist_ok=True)
os.makedirs(summary_path, exist_ok=True)

# Initialize a DataFrame for the summary sheet
summary_columns = [
    "Month_year", "RF_Lag1", "R(1)", "R(2)", "R(3)", "R(4)", "R(5)",
    "B(1)", "B(2)", "B(3)", "B(4)", "B(5)", "R(L)", "R(H)", "B(L)", "B(H)", "r_BAB"
]
summary_df = pd.DataFrame(columns=summary_columns)

# Process each portfolio file
for file in os.listdir(portfolio_path):
    if file.startswith("PORT_") and file.endswith(".csv"):
        # Load the data
        file_path = os.path.join(portfolio_path, file)
        df = pd.read_csv(file_path)
        
        # Extract Month_Year for tracking
        month_year = file.split("_")[1] + "-" + file.split("_")[2].replace(".csv", "")
        
        # Function 1: Beta Sorting
        df['Beta_rank'] = df['Adjusted_Beta'].rank(method='min')
        df['Beta_Quintile'] = pd.qcut(df['Beta_rank'], 5, labels=[1, 2, 3, 4, 5])
        
        quintile_avg_return = df.groupby('Beta_Quintile', observed=False)['ROI_Lag1'].mean()
        quintile_avg_beta = df.groupby('Beta_Quintile', observed=False)['Adjusted_Beta'].mean()
        
        df['Return_Quintile'] = df['Beta_Quintile'].map(quintile_avg_return)
        df['Avg Beta'] = df['Beta_Quintile'].map(quintile_avg_beta)
        
        # Function 2: Split Beta
        median_rank = df['Beta_rank'].median()
        df['BAB port'] = df['Beta_rank'].apply(lambda x: 'L' if x < median_rank else ('H' if x > median_rank else None))
        
        avg_z = df['Beta_rank'].mean()
        df['NormalizeConstant'] = 2 / sum(abs(df['Beta_rank'] - avg_z))
        df['Weight'] = df['NormalizeConstant'] * abs(df['Beta_rank'] - avg_z)
        
        # Weighted calculations for return and beta
        df['Weighted_Return'] = df['ROI_Lag1'] * df['Weight']
        df['Weighted_Beta'] = df['Adjusted_Beta'] * df['Weight']
        
        r_H = df.loc[df['BAB port'] == 'H', 'Weighted_Return'].sum()
        r_L = df.loc[df['BAB port'] == 'L', 'Weighted_Return'].sum()
        B_H = df.loc[df['BAB port'] == 'H', 'Weighted_Beta'].sum()
        B_L = df.loc[df['BAB port'] == 'L', 'Weighted_Beta'].sum()
        
        rf = df['RF_Lag1'].iloc[0]
        r_BAB = ((1 / B_L) * (r_L - rf)) - ((1 / B_H) * (r_H - rf))
        
        # Append summary data
        summary_row = pd.DataFrame([{
            "Month_year": month_year,
            "RF_Lag1": rf,
            "R(1)": quintile_avg_return[1],
            "R(2)": quintile_avg_return[2],
            "R(3)": quintile_avg_return[3],
            "R(4)": quintile_avg_return[4],
            "R(5)": quintile_avg_return[5],
            "B(1)": quintile_avg_beta[1],
            "B(2)": quintile_avg_beta[2],
            "B(3)": quintile_avg_beta[3],
            "B(4)": quintile_avg_beta[4],
            "B(5)": quintile_avg_beta[5],
            "R(L)": r_L,
            "R(H)": r_H,
            "B(L)": B_L,
            "B(H)": B_H,
            "r_BAB": r_BAB
        }])
        summary_df = pd.concat([summary_df, summary_row], ignore_index=True)
        
        # Save the processed file
        output_file_path = os.path.join(output_path, f"Processed_{file}")
        df.to_csv(output_file_path, index=False)

# Save the summary sheet
summary_file_path = os.path.join(summary_path, "BAC_Summary.csv")
summary_df.to_csv(summary_file_path, index=False)

print("Processing and summary completed. Files are saved in the respective output directories.")


C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_26488\2904403981.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_df = pd.concat([summary_df, summary_row], ignore_index=True)


Processing and summary completed. Files are saved in the respective output directories.


In [2]:
import os
import pandas as pd

# Input and output paths
portfolio_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\01_Input\06_PortFolio\05_Port_Month_Year_FullData"
output_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\02_Output\13_BAB_New_Value_Weight\Data"
summary_path = r"C:\Users\Kantinant Sukkert\Desktop\BAC Thematic\02_Output\13_BAB_New_Value_Weight\Summary"

# Ensure output directories exist
os.makedirs(output_path, exist_ok=True)
os.makedirs(summary_path, exist_ok=True)

# Initialize a DataFrame for the summary sheet
summary_columns = [
    "Month_year", "Decimal_TBILL", "R(L2)", "R(H2)", "B(L2)", "B(H2)", "r_BAB"
]
summary_df = pd.DataFrame(columns=summary_columns)

# Process each portfolio file
for file in os.listdir(portfolio_path):
    if file.startswith("PORT_") and file.endswith(".csv"):
        # Load the data
        file_path = os.path.join(portfolio_path, file)
        df = pd.read_csv(file_path)
        
        # Extract Month_Year for tracking
        month_year = file.split("_")[1] + "-" + file.split("_")[2].replace(".csv", "")
        
        # Median-based port split
        median_rank = df['Adjusted_Beta'].median()
        df['BAB port'] = df['Adjusted_Beta'].apply(lambda x: 'L' if x < median_rank else 'H')
        
        # Sum of Beta and inverse Beta calculations
        sum_beta_h = df.loc[df['BAB port'] == 'H', 'Adjusted_Beta'].sum()
        sum_inv_beta_l = (1 / df.loc[df['BAB port'] == 'L', 'Adjusted_Beta']).sum()

        # Assign weights based on BAB port
        df['SumBeta'] = df['BAB port'].map({'H': sum_beta_h, 'L': sum_inv_beta_l})
        df['WeightValue'] = df.apply(
            lambda row: row['Adjusted_Beta'] / row['SumBeta']
            if row['BAB port'] == 'H' else (1 / row['Adjusted_Beta']) / row['SumBeta'],
            axis=1
        )
        
        # Weighted return and beta
        df['Weighted_Return2'] = df['ROI_Lag1'] * df['WeightValue']
        df['Weighted_Beta2'] = df['Adjusted_Beta'] * df['WeightValue']
        
        # Calculate R(L2), R(H2), B(L2), B(H2)
        r_l2 = df.loc[df['BAB port'] == 'L', 'Weighted_Return2'].sum()
        r_h2 = df.loc[df['BAB port'] == 'H', 'Weighted_Return2'].sum()
        b_l2 = df.loc[df['BAB port'] == 'L', 'Weighted_Beta2'].sum()
        b_h2 = df.loc[df['BAB port'] == 'H', 'Weighted_Beta2'].sum()
        
        # Risk-free rate
        rf = df['RF_Lag1'].iloc[0]
        
        # BAB return
        r_bab = ((1 / b_l2) * (r_l2 - rf)) - ((1 / b_h2) * (r_h2 - rf))
        
        # Append summary data
        summary_row = pd.DataFrame([{
            "Month_year": month_year,
            "RF_Lag1": rf,
            "R(L2)": r_l2,
            "R(H2)": r_h2,
            "B(L2)": b_l2,
            "B(H2)": b_h2,
            "r_BAB": r_bab
        }])
        summary_df = pd.concat([summary_df, summary_row], ignore_index=True)
        
        # Save the processed file
        output_file_path = os.path.join(output_path, f"Processed_{file}")
        df.to_csv(output_file_path, index=False)

# Save the summary sheet
summary_file_path = os.path.join(summary_path, "BAB_Summary.csv")
summary_df.to_csv(summary_file_path, index=False)

print("Processing and summary completed. Files are saved in the respective output directories.")


C:\Users\Kantinant Sukkert\AppData\Local\Temp\ipykernel_3772\978345871.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_df = pd.concat([summary_df, summary_row], ignore_index=True)


Processing and summary completed. Files are saved in the respective output directories.
